# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.34.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'coupon-reco-automl'
project_folder = './coupon-reco-proj-automl'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
coupon-reco-automl,quick-starts-ws-162940,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cc-sta-ds12-2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           min_nodes=1,
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# NOTE: update the key to match the dataset name
found = False
key = "in-vehicle-coupon-reco"
description_text = "In-vehicle coupon recommendation Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21.0,Unmarried partner,...,never,None,4~8,1~3,1,1,0,0,1,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
# Performing EDA, we see that the class is balanced one.

automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    'model_explainability': True
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
coupon-reco-automl,AutoML_e21e1292-fbc5-4445-8213-bd4c24dd2fdf,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output = True)
assert(remote_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
coupon-reco-automl,AutoML_e21e1292-fbc5-4445-8213-bd4c24dd2fdf,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
# Retrieve and save your best automl model.
automl_best_run, automl_fitted_model = remote_run.get_output()


# Reference: https://knowledge.udacity.com/questions/357775 
print("Fitted model: ", automl_fitted_model.steps[-1])
print("****"*27)
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("****"*27)
print("AutoML run Summary: ", remote_run.summary())

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

Fitted model:  ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('28', Pipeline(
        memory=None,
        steps=[('sparsenormalizer', Normalizer(
            copy=True,
            norm='max'
        )), ('xgboostclassifier', XGBoostClassifier(
            random_state=0,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            ),
            booster='gbtree',
            colsample_bytree=0.7,
            eta=0.3,
            gamma=0,
            max_depth=9,
            max_leaves=0,
            n_estimators=400,
            objective='reg:logistic',
            reg_alpha=0,
            reg_lambda=1.3541666666666667,
            subsample=1,
            tree_method='auto'
        ))],
        verbose=False
    )), ('33', Pipeline(
        memory=None,
        steps=[('sparsenormalizer', Normalizer(
            copy=True,
            norm='l2'
        )), ('xgboost

In [13]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
coupon-reco-automl,AutoML_e21e1292-fbc5-4445-8213-bd4c24dd2fdf_47,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
#TODO: Save the best model
best_model, model = remote_run.get_output()
model = best_model.register_model(model_name='automl-best-model',model_path='outputs/model.pkl')

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig, Model
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
# model = automl_best_run.register_model(model_name = "mymodel", model_path = "outputs/model.pkl")
automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'condaEnv.yml')

env = Environment.from_conda_specification('condaEnv', 'condaEnv.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True,tags = {'name':'Action Prediction'},description='In-vehicle Coupon Recommendation')
service = model.deploy(workspace=ws,name="in-vehicle-coupon-reco-automl-deploy",models=[model],inference_config=inference_config,deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [ ]:
service.update(enable_app_insights=True)

print(service.state)
print("scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json
import requests
# URL for the web service
scoring_uri = 'http://617ba078-e419-496f-b992-a73545385b31.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'QSezHp5JMibUTuMLORyaLju41EYcjyBA'

data = {
  "data": [
    {#ALLOW
                  'Source Port': 50148, 
                  'Destination Port': 80, 
                  'NAT Source Port': 57004,
                  'NAT Destination Port': 80,
                  'Bytes': 9058,
                  'Bytes Sent': 868,
                  'Bytes Received': 8190,
                  'Packets': 18,
                  'Elapsed Time (sec)': 46,
                  'pkts_sent': 10,
                  'pkts_received': 8
                 
                 
    },
    {#DENY
                  'Source Port': 56103, 
                  'Destination Port': 21247, 
                  'NAT Source Port': 0,
                  'NAT Destination Port': 0,
                  'Bytes': 66,
                  'Bytes Sent': 66,
                  'Bytes Received': 0,
                  'Packets': 1,
                  'Elapsed Time (sec)': 0,
                  'pkts_sent': 1,
                  'pkts_received': 0
                 
                 
    },
    {#DROP
                  'Source Port': 49783, 
                  'Destination Port': 445, 
                  'NAT Source Port': 0,
                  'NAT Destination Port': 0,
                  'Bytes': 70,
                  'Bytes Sent': 70,
                  'Bytes Received': 0,
                  'Packets': 1,
                  'Elapsed Time (sec)': 0,
                  'pkts_sent': 1,
                  'pkts_received': 0
                 
                 
    }
 ]
}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)
#print(f"Predicted: {output}\nExpected: {y}")

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction ", resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#print the logs of the web service
web_service_logs = service.get_logs()
with open("./web_service_logs.txt", 'w') as output:
    output.write(str(web_service_logs))

print(web_service_logs)


In [ ]:
#Deleting Service
service.delete()
compute_target.delete()
compute_target.wait_for_completion(show_output=True, is_delete_operation=True)

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
